In [ ]:
import pandas as pd
import requests, zipfile, StringIO
pd.set_option('display.max_columns',200)
import numpy as np
import grequests
from lxml import html
import matplotlib.pyplot as plt
%matplotlib inline

# ACS Population Data

In [ ]:
acs5yr_base = 'https://www2.census.gov/programs-surveys/acs/data/pums/2016/5-Year/'

In [ ]:
def pull_acsfile(state, filetype='csv', ph='p'):
    r = requests.get(
        acs5yr_base + '{filetype}_{ph}{state}.zip'.format(filetype=filetype,
                                                          ph=ph,
                                                          state=state),
            stream=True)
    z = zipfile.ZipFile(StringIO.StringIO(r.content))
    z.extractall()

In [ ]:
statelist = ['il','ca','ny']

In [ ]:
for state in statelist:
    print('{} Started'.format(state))
    pull_acsfile(state)
    print('{} Complete'.format(state))

# Zip Code Population Data (Incremental pop data)

In [ ]:
# r = requests.get('https://www.kaggle.com/census/us-population-by-zip-code/downloads/us-population-by-zip-code.zip', stream=True)
# z = zipfile.ZipFile(StringIO.StringIO(r.content))
# z.extractall()

pop location
https://www.kaggle.com/census/us-population-by-zip-code/downloads/us-population-by-zip-code.zip

In [ ]:
df = pd.read_csv('ss16pca.csv')
fig, ax = plt.subplots(figsize=(20,5), dpi=200)
ax.hist(df['AGEP'],bins=94)

In [ ]:
age_wage = df.dropna(subset=['AGEP','WAGP']).groupby('AGEP').agg({'WAGP':'mean'})

age_wage.plot(kind='bar',figsize=(20,5))

# BART Locations

In [ ]:
r = requests.get('https://www.bart.gov/stations')
tree = htmltree = html.fromstring(r.content)
t = tree.xpath('//*[@id="station-lists"]/ul/li/a')

In [ ]:
stations = []
for list_obj in t:
#     try:
    station_url = list_obj.items()[0][1]
    stations.append(station_url)
#     except:
#         pass

In [ ]:
zipcodes = []
station_addrs = []
for sta_url in stations:
    full_url = 'https://www.bart.gov' + sta_url
    sta_r = requests.get(full_url)
    tree = htmltree = html.fromstring(sta_r.content)
    address = tree.xpath('//*[@id="main-content"]/div[1]/div/span[1]/text()')[0]
    city = tree.xpath('//*[@id="main-content"]/div[1]/div/span[2]/text()')[0]
    zipcode = tree.xpath('//*[@id="main-content"]/div[1]/div/span[4]/text()')[0]
    zipcodes.append(zipcode)
    station_addrs.append('{}, {}, CA, {}'.format(address,city,zipcode))

In [ ]:
def get_latlon(address):
    address = address.replace(' ','+').strip("'")
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={}, CA'.format(address))
    resp_json_payload = response.json()
    try:
        return resp_json_payload['results'][0]['geometry']['location']
    except:
        try:
            response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={}, CA'.format(address))
            resp_json_payload = response.json()
            return resp_json_payload['results'][0]['geometry']['location']
        except:
            return address #resp_json_payload['results']

In [ ]:
stations_ = {}
for add_ in station_addrs:
    payload = get_latlon(add_)
    stations_.update({add_:payload})

stations_2=[]
for stat, loc_ in stations_.items():
    try:
        lat, lng = loc_['lat'], loc_['lng']
    except:
        lat, lng = np.nan, np.nan
    stations_2.append({'station':stat,
                       'latatude':lat,
                       'longitude':lng,
                       'raw':loc_})

sta_loc_df = pd.from_dict(stations_2)
sta_loc_df.to_csv('station_locations.csv', index=False)

In [ ]:
df = pd.read_csv('station_locations.csv')
del df['Unnamed: 0']

In [ ]:
df

## GEODATABSE Integration
Link: https://www.census.gov/geo/maps-data/data/tiger-data.html (Download Block Group level for a given state)

Documentation: https://www2.census.gov/geo/tiger/TIGER_DP/2016ACS/Metadata/BG_METADATA_2016.txt